## Perubahan Struktur Dataset Stok Obat


Perubahan Struktur Dataset Stok

In [1]:
import pandas as pd
import re
from pathlib import Path

# === 1. Baca Dataset ===
file_path = Path("stok_final_fix.csv")
df = pd.read_csv(file_path)

print("Sebelum pembersihan:")
print(df.head(), "\n")

# === 2. Standarisasi Format ===

# 2.1 Bersihkan spasi berlebih & kapitalisasi nama kolom
df.columns = [col.strip().upper().replace(" ", "_") for col in df.columns]

# 2.2 Standarkan teks (misalnya nama obat)
if "NAMA_OBAT" in df.columns:
    df["NAMA_OBAT"] = (
        df["NAMA_OBAT"]
        .astype(str)
        .str.strip()
        .str.title()
        .replace("-", " ")
    )

# 2.3 Konversi kolom harga/stok menjadi numerik
def parse_number(x):
    if pd.isna(x) or str(x).strip() == "":
        return 0.0
    x = str(x).replace(".", "").replace(",", ".").replace("Rp", "").replace(" ", "")
    try:
        return float(x)
    except:
        return 0.0

for col in df.columns:
    if any(kata in col for kata in ["HARGA", "NILAI", "STOK", "JUMLAH"]):
        df[col] = df[col].apply(parse_number)

# === 3. Validasi Format Tanggal dan Kode ===

# 3.1 Format tanggal
if "TANGGAL" in df.columns:
    df["TANGGAL"] = pd.to_datetime(df["TANGGAL"], errors="coerce", dayfirst=True)

# 3.2 Validasi kode obat (contoh pola: huruf diikuti angka, seperti OB123)
if "KODE_OBAT" in df.columns:
    df["VALID_KODE"] = df["KODE_OBAT"].apply(lambda x: bool(re.match(r"^[A-Z]+\d+$", str(x).strip())))

# === 4. Rule-Based & Constraint-Based Validation ===

# 4.1 Nilai stok dan harga tidak boleh negatif
for col in df.columns:
    if any(kata in col for kata in ["STOK", "HARGA", "NILAI"]):
        df.loc[df[col] < 0, "FLAG_ERROR"] = "NEGATIVE VALUE"

# 4.2 Tanggal tidak boleh di masa depan
if "TANGGAL" in df.columns:
    today = pd.Timestamp.now().normalize()
    df.loc[df["TANGGAL"] > today, "FLAG_ERROR"] = "INVALID DATE"

# === 5. Pattern-Based / Statistical Detection ===
# Deteksi outlier harga dengan z-score sederhana
import numpy as np

if "HARGA" in df.columns:
    mean = df["HARGA"].mean()
    std = df["HARGA"].std()
    df["Z_SCORE_HARGA"] = (df["HARGA"] - mean) / std
    df.loc[df["Z_SCORE_HARGA"].abs() > 3, "FLAG_ERROR"] = "OUTLIER"

# === 6. Simpan Hasil Akhir ===
output_path = Path("stok_final_fix_cleaned.xlsx")
df.to_excel(output_path, index=False)

print("✅ Data berhasil dibersihkan dan disimpan ke:", output_path)
print(df.head(10))


Sebelum pembersihan:
      KODE         NAMA_PRODUK LOKASI  QTY_STOK   UNIT
0  A000001          ANATON TAB   ETL1        12  STRIP
1   A00001       ACTIVED HIJAU  ETL3A         2    BTL
2  A000012  APIALYS SYR 100 ML  ETL3A         2    BTL
3  A000014     ALKOHOL 1000 ML  ETL3B         7    BTL
4  A000016     ALLOPURINOL 300   RAK2        40  STRIP 

✅ Data berhasil dibersihkan dan disimpan ke: stok_final_fix_cleaned.xlsx
      KODE         NAMA_PRODUK LOKASI  QTY_STOK   UNIT FLAG_ERROR
0  A000001          ANATON TAB   ETL1      12.0  STRIP        NaN
1   A00001       ACTIVED HIJAU  ETL3A       2.0    BTL        NaN
2  A000012  APIALYS SYR 100 ML  ETL3A       2.0    BTL        NaN
3  A000014     ALKOHOL 1000 ML  ETL3B       7.0    BTL        NaN
4  A000016     ALLOPURINOL 300   RAK2      40.0  STRIP        NaN
5  A000018   ATORVASTATIN 10MG   RAK2       6.0  STRIP        NaN
6   A00004     ACYCLOVIR 200MG   RAK2      13.0  STRIP        NaN
7  A000040         MEFIX 500MG   RAK1       9.

### Parsing Data Stok

In [46]:
import pandas as pd
import re
from pathlib import Path
import numpy as np

# === 1. Baca file hasil parsing sebelumnya ===
file_path = Path("stok_final_fix.csv")
df = pd.read_csv(file_path)

print("🔹 Data awal:")
print(df.head(), "\n")

# === 2. Standarisasi format teks & angka ===

# 2.1 Pastikan nama kolom huruf besar semua dan tanpa spasi
df.columns = [col.strip().upper().replace(" ", "_") for col in df.columns]

# 2.2 Standarkan format teks
if "NAMA_PRODUK" in df.columns:
    df["NAMA_PRODUK"] = (
        df["NAMA_PRODUK"]
        .astype(str)
        .str.strip()
        .str.title()         # Kapital di awal kata
        .str.replace(r"\s+", " ", regex=True)
    )

if "UNIT" in df.columns:
    df["UNIT"] = df["UNIT"].str.upper().str.strip()

if "LOKASI" in df.columns:
    df["LOKASI"] = df["LOKASI"].str.upper().str.strip()

# 2.3 Pastikan nilai stok numerik
def parse_number(x):
    if pd.isna(x) or str(x).strip() == "":
        return 0.0
    x = str(x).replace(".", "").replace(",", ".")
    try:
        return float(x)
    except:
        return 0.0

if "QTY_STOK" in df.columns:
    df["QTY_STOK"] = df["QTY_STOK"].apply(parse_number)

# === 3. Validasi format & logika ===

# 3.1 Validasi kode produk (harus huruf + angka)
if "KODE" in df.columns:
    df["VALID_KODE"] = df["KODE"].apply(lambda x: bool(re.match(r"^[A-Z]+\d+$", str(x).strip())))

# 3.2 Deteksi nilai stok tidak wajar (misal negatif atau terlalu besar)
df["FLAG_ERROR"] = ""

if "QTY_STOK" in df.columns:
    df.loc[df["QTY_STOK"] < 0, "FLAG_ERROR"] = "STOK NEGATIF"
    df.loc[df["QTY_STOK"] > df["QTY_STOK"].mean() + 3 * df["QTY_STOK"].std(), "FLAG_ERROR"] = "OUTLIER"

# === 4. Constraint & Pattern-Based Detection ===

# 4.1 Cek unit yang tidak sesuai format umum
valid_units = ["STRIP", "BTL", "BOX", "PCS"]
if "UNIT" in df.columns:
    df.loc[~df["UNIT"].isin(valid_units), "FLAG_ERROR"] = "UNIT TIDAK VALID"

# 4.2 Cek lokasi kosong
if "LOKASI" in df.columns:
    df.loc[df["LOKASI"] == "", "FLAG_ERROR"] = "LOKASI KOSONG"

# === 5. Cross-Dataset Consistency (opsional)
# Jika nanti kamu punya file stok lain (misalnya stok gudang vs stok apotek),
# di sini bisa dibandingkan konsistensinya.

# === 6. Simpan hasil akhir ===
output_path = Path("stok_final_validated.xlsx")
df.to_excel(output_path, index=False)

print("✅ File berhasil dibersihkan dan divalidasi!")
print("📁 Hasil disimpan di:", output_path)
print(df.head(15))


🔹 Data awal:
      KODE         NAMA_PRODUK LOKASI  QTY_STOK   UNIT
0  A000001          ANATON TAB   ETL1        12  STRIP
1   A00001       ACTIVED HIJAU  ETL3A         2    BTL
2  A000012  APIALYS SYR 100 ML  ETL3A         2    BTL
3  A000014     ALKOHOL 1000 ML  ETL3B         7    BTL
4  A000016     ALLOPURINOL 300   RAK2        40  STRIP 

✅ File berhasil dibersihkan dan divalidasi!
📁 Hasil disimpan di: stok_final_validated.xlsx
       KODE         NAMA_PRODUK LOKASI  QTY_STOK   UNIT  VALID_KODE  \
0   A000001          Anaton Tab   ETL1      12.0  STRIP        True   
1    A00001       Actived Hijau  ETL3A       2.0    BTL        True   
2   A000012  Apialys Syr 100 Ml  ETL3A       2.0    BTL        True   
3   A000014     Alkohol 1000 Ml  ETL3B       7.0    BTL        True   
4   A000016     Allopurinol 300   RAK2      40.0  STRIP        True   
5   A000018   Atorvastatin 10Mg   RAK2       6.0  STRIP        True   
6    A00004     Acyclovir 200Mg   RAK2      13.0  STRIP        True

In [ ]:
import pandas as pd
import re
from pathlib import Path
import numpy as np

# === 1. Baca file hasil parsing sebelumnya ===
file_path = Path("stok_final_fix.csv")
df = pd.read_csv(file_path)

print("🔹 Data awal:")
print(df.head(), "\n")

# === 2. Standarisasi format teks & angka ===

# 2.1 Pastikan nama kolom huruf besar semua dan tanpa spasi
df.columns = [col.strip().upper().replace(" ", "_") for col in df.columns]

# 2.2 Standarkan format teks
if "NAMA_PRODUK" in df.columns:
    df["NAMA_PRODUK"] = (
        df["NAMA_PRODUK"]
        .astype(str)
        .str.strip()
        .str.title()         # Kapital di awal kata
        .str.replace(r"\s+", " ", regex=True)
    )

if "UNIT" in df.columns:
    df["UNIT"] = df["UNIT"].str.upper().str.strip()

if "LOKASI" in df.columns:
    df["LOKASI"] = df["LOKASI"].str.upper().str.strip()

# 2.3 Pastikan nilai stok numerik
def parse_number(x):
    if pd.isna(x) or str(x).strip() == "":
        return 0.0
    x = str(x).replace(".", "").replace(",", ".")
    try:
        return float(x)
    except:
        return 0.0

if "QTY_STOK" in df.columns:
    df["QTY_STOK"] = df["QTY_STOK"].apply(parse_number)

# === 3. Validasi format & logika ===

# 3.1 Validasi kode produk (harus huruf + angka)
if "KODE" in df.columns:
    df["VALID_KODE"] = df["KODE"].apply(lambda x: bool(re.match(r"^[A-Z]+\d+$", str(x).strip())))

# 3.2 Deteksi nilai stok tidak wajar (misal negatif atau terlalu besar)
df["FLAG_ERROR"] = ""

if "QTY_STOK" in df.columns:
    df.loc[df["QTY_STOK"] < 0, "FLAG_ERROR"] = "STOK NEGATIF"
    df.loc[df["QTY_STOK"] > df["QTY_STOK"].mean() + 3 * df["QTY_STOK"].std(), "FLAG_ERROR"] = "OUTLIER"

# === 4. Constraint & Pattern-Based Detection ===

# 4.1 Cek unit yang tidak sesuai format umum
valid_units = ["STRIP", "BTL", "BOX", "PCS"]
if "UNIT" in df.columns:
    df.loc[~df["UNIT"].isin(valid_units), "FLAG_ERROR"] = "UNIT TIDAK VALID"

# 4.2 Cek lokasi kosong
if "LOKASI" in df.columns:
    df.loc[df["LOKASI"] == "", "FLAG_ERROR"] = "LOKASI KOSONG"

# === 5. Cross-Dataset Consistency (opsional)
# Jika nanti kamu punya file stok lain (misalnya stok gudang vs stok apotek),
# di sini bisa dibandingkan konsistensinya.

# === 6. Simpan hasil akhir ===
output_path = Path("stok_final_validated.xlsx")
df.to_excel(output_path, index=False)

print("✅ File berhasil dibersihkan dan divalidasi!")
print("📁 Hasil disimpan di:", output_path)
print(df.head(15))


🔹 Data awal:
      KODE         NAMA_PRODUK LOKASI  QTY_STOK   UNIT
0  A000001          ANATON TAB   ETL1        12  STRIP
1   A00001       ACTIVED HIJAU  ETL3A         2    BTL
2  A000012  APIALYS SYR 100 ML  ETL3A         2    BTL
3  A000014     ALKOHOL 1000 ML  ETL3B         7    BTL
4  A000016     ALLOPURINOL 300   RAK2        40  STRIP 

✅ File berhasil dibersihkan dan divalidasi!
📁 Hasil disimpan di: stok_final_validated.xlsx
       KODE         NAMA_PRODUK LOKASI  QTY_STOK   UNIT  VALID_KODE  \
0   A000001          Anaton Tab   ETL1      12.0  STRIP        True   
1    A00001       Actived Hijau  ETL3A       2.0    BTL        True   
2   A000012  Apialys Syr 100 Ml  ETL3A       2.0    BTL        True   
3   A000014     Alkohol 1000 Ml  ETL3B       7.0    BTL        True   
4   A000016     Allopurinol 300   RAK2      40.0  STRIP        True   
5   A000018   Atorvastatin 10Mg   RAK2       6.0  STRIP        True   
6    A00004     Acyclovir 200Mg   RAK2      13.0  STRIP        True

In [3]:
print("📊 === RINGKASAN DATAFRAME ===")

print("\n📌 Informasi Struktur DataFrame:")
df.info()

print("\n📌 Tipe Data Tiap Kolom:")
print(df.dtypes)

print("\n📌 Jumlah Missing Value Tiap Kolom:")
print(df.isna().sum())

print("\n📌 Statistik Deskriptif untuk Kolom Numerik:")
print(df.describe())

print("\n📌 5 Baris Data Pertama:")
print(df.head())

📊 === RINGKASAN DATAFRAME ===

📌 Informasi Struktur DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1513 entries, 0 to 1512
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   KODE         1513 non-null   object 
 1   NAMA_PRODUK  1513 non-null   object 
 2   LOKASI       1513 non-null   object 
 3   QTY_STOK     1513 non-null   float64
 4   UNIT         1513 non-null   object 
 5   VALID_KODE   1513 non-null   bool   
 6   FLAG_ERROR   1513 non-null   object 
dtypes: bool(1), float64(1), object(5)
memory usage: 72.5+ KB

📌 Tipe Data Tiap Kolom:
KODE            object
NAMA_PRODUK     object
LOKASI          object
QTY_STOK       float64
UNIT            object
VALID_KODE        bool
FLAG_ERROR      object
dtype: object

📌 Jumlah Missing Value Tiap Kolom:
KODE           0
NAMA_PRODUK    0
LOKASI         0
QTY_STOK       0
UNIT           0
VALID_KODE     0
FLAG_ERROR     0
dtype: int64

📌 Statistik Deskriptif

In [7]:
##Konversi tipe data sesuai isi dataset ===

# Konversi kolom numerik (stok)
if 'QTY_STOK' in df.columns:
    # Ganti koma dengan titik, hilangkan spasi
    df['QTY_STOK'] = (
        df['QTY_STOK']
        .astype(str)
        .str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
    )
    df['QTY_STOK'] = pd.to_numeric(df['QTY_STOK'], errors='coerce')

# Konversi kolom teks
text_columns = ['KODE', 'NAMA_PRODUK', 'LOKASI', 'UNIT']
for col in text_columns:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.strip()                 # hapus spasi depan/belakang
            .str.replace(r'\s+', ' ', regex=True)  # rapikan spasi ganda
            .str.upper()                 # standarkan jadi huruf besar
        )
# === 3. Tampilkan hasil konversi ===
print("📊 === TIPE DATA SETELAH KONVERSI ===")
print(df.dtypes)


📊 === TIPE DATA SETELAH KONVERSI ===
KODE           object
NAMA_PRODUK    object
LOKASI         object
QTY_STOK        int64
UNIT           object
VALID_KODE       bool
FLAG_ERROR     object
dtype: object


In [8]:
# Cari baris yang memiliki nama produk ganda (duplikat)
duplikat_produk = df[df.duplicated(subset=['NAMA_PRODUK'], keep=False)]

# Jumlah duplikasi
print(f"📌 Jumlah duplikasi berdasarkan NAMA_PRODUK : {duplikat_produk.shape[0]}")

# Tampilkan sebagian contoh duplikasi
if not duplikat_produk.empty:
    print("\n📋 Contoh data duplikat berdasarkan NAMA_PRODUK:")
    print(duplikat_produk.sort_values('NAMA_PRODUK').head(10))
else:
    print("\n✅ Tidak ditemukan data duplikat berdasarkan NAMA_PRODUK.")

📌 Jumlah duplikasi berdasarkan NAMA_PRODUK : 0

✅ Tidak ditemukan data duplikat berdasarkan NAMA_PRODUK.


Tidak melakukan penghapusan duplikasi

## Deteksi Inkonsistensi Nama Produk per Kode


In [9]:
print("📋 Kolom sebelum normalisasi:")
print(df.columns.tolist())

df.columns = df.columns.str.strip().str.upper()

print("\n📋 Kolom setelah normalisasi:")
print(df.columns.tolist())

# Group berdasarkan kode, lalu hitung jumlah nama produk unik per kode
kode_nama_group = df.groupby('KODE')['NAMA_PRODUK'].nunique()

# Ambil hanya kode yang memiliki lebih dari 1 nama produk berbeda
kode_nama_tidak_konsisten = kode_nama_group[kode_nama_group > 1]

print(f"\n📌 Jumlah kode dengan nama produk tidak konsisten: {len(kode_nama_tidak_konsisten)}")

# Tampilkan kode dan variasi nama produknya
for kode in kode_nama_tidak_konsisten.index:
    print(f"\n🔎 Kode {kode} memiliki beberapa nama produk:")
    print(df[df['KODE'] == kode]['NAMA_PRODUK'].unique())

📋 Kolom sebelum normalisasi:
['KODE', 'NAMA_PRODUK', 'LOKASI', 'QTY_STOK', 'UNIT', 'VALID_KODE', 'FLAG_ERROR']

📋 Kolom setelah normalisasi:
['KODE', 'NAMA_PRODUK', 'LOKASI', 'QTY_STOK', 'UNIT', 'VALID_KODE', 'FLAG_ERROR']

📌 Jumlah kode dengan nama produk tidak konsisten: 0


In [10]:
# === 6. Rekapitulasi Data Berdasarkan Dataset ===

# Grouping dan agregasi data stok
rekap = df.groupby(['KODE', 'NAMA_PRODUK', 'LOKASI', 'UNIT']).agg({
    'QTY_STOK': 'sum'
}).reset_index()

print("📊 Rekapitulasi Data (Top 10):")
print(rekap.head(10))

📊 Rekapitulasi Data (Top 10):
      KODE         NAMA_PRODUK LOKASI   UNIT  QTY_STOK
0  A000001          ANATON TAB   ETL1  STRIP       120
1   A00001       ACTIVED HIJAU  ETL3A    BTL        20
2  A000012  APIALYS SYR 100 ML  ETL3A    BTL        20
3  A000014     ALKOHOL 1000 ML  ETL3B    BTL        70
4  A000016     ALLOPURINOL 300   RAK2  STRIP       400
5  A000018   ATORVASTATIN 10MG   RAK2  STRIP        60
6   A00004     ACYCLOVIR 200MG   RAK2  STRIP       130
7  A000040         MEFIX 500MG   RAK1  STRIP        90
8   A00005     ACYCLOVIR 400MG   RAK2  STRIP       210
9  A000066       ANDALAN KB FE   RAK4  STRIP       350


In [11]:
# === 7. Standarisasi Format Teks dan Satuan Numerik ===

import re

# Kolom teks dan numerik yang relevan untuk dataset kamu
text_cols = ['KODE', 'NAMA_PRODUK', 'LOKASI', 'UNIT']
num_cols = ['QTY_STOK']

# 🔢 Standarisasi kolom numerik (hilangkan karakter aneh seperti titik/koma)
for col in num_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(',', '.', regex=False)          # ganti koma jadi titik desimal
        .str.replace(r'[^\d\.]', '', regex=True)     # hapus karakter non-numerik
    )
    df[col] = pd.to_numeric(df[col], errors='coerce').round(2)

# 🔤 Standarisasi teks: kapitalisasi, hilangkan spasi berlebih, karakter asing
for col in text_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()
        .str.replace(r'\s+', ' ', regex=True)  # hilangkan spasi ganda
        .str.upper()
    )

# 🔍 Standarisasi format kode produk (hanya huruf besar dan angka)
df['KODE'] = (
    df['KODE']
    .astype(str)
    .str.strip()
    .str.replace(r'[^A-Z0-9]', '', regex=True)   # hanya huruf & angka
    .str.upper()
)

# ✅ Validasi format kode: panjang 3–12 karakter, hanya huruf besar & angka
def valid_kode(k):
    return bool(re.match(r'^[A-Z0-9]{3,12}$', str(k).strip()))

df['KODE_VALID'] = df['KODE'].apply(valid_kode)

# Tampilkan hasil standarisasi
print("📋 Contoh hasil standarisasi data:")
print(df.head(10))


📋 Contoh hasil standarisasi data:
      KODE         NAMA_PRODUK LOKASI  QTY_STOK   UNIT  VALID_KODE FLAG_ERROR  \
0  A000001          ANATON TAB   ETL1       120  STRIP        True              
1   A00001       ACTIVED HIJAU  ETL3A        20    BTL        True              
2  A000012  APIALYS SYR 100 ML  ETL3A        20    BTL        True              
3  A000014     ALKOHOL 1000 ML  ETL3B        70    BTL        True              
4  A000016     ALLOPURINOL 300   RAK2       400  STRIP        True              
5  A000018   ATORVASTATIN 10MG   RAK2        60  STRIP        True              
6   A00004     ACYCLOVIR 200MG   RAK2       130  STRIP        True              
7  A000040         MEFIX 500MG   RAK1        90  STRIP        True              
8   A00005     ACYCLOVIR 400MG   RAK2       210  STRIP        True              
9  A000066       ANDALAN KB FE   RAK4       350  STRIP        True              

   KODE_VALID  
0        True  
1        True  
2        True  
3        T

#### RULE-BASED METHOD

In [12]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

In [13]:
# Flag kondisi anomali
df['RULE_FLAG'] = np.where(
    (df['QTY_STOK'] <= 0) |                               # stok tidak boleh nol atau negatif
    (df['KODE'].isna()) | (df['KODE'].str.strip() == '') | # kode kosong
    (df['NAMA_PRODUK'].isna()) | (df['NAMA_PRODUK'].str.strip() == '') |  # nama produk kosong
    (df['LOKASI'].isna()) | (df['LOKASI'].str.strip() == '') |            # lokasi kosong
    (df['UNIT'].isna()) | (df['UNIT'].str.strip() == ''),                 # unit kosong
    True, False
)

# Hitung jumlah anomali
rule_count = df['RULE_FLAG'].sum()
print(f"🧩 [Rule-Based Detection] Ditemukan {rule_count} baris anomali berdasarkan aturan logika.")

# Tampilkan contoh baris anomali (maksimal 5 baris pertama)
if rule_count > 0:
    print(df[df['RULE_FLAG']].head(5))
else:
    print("✅ Tidak ditemukan anomali pada dataset stok.")

print("--------------------------------------------------\n")

🧩 [Rule-Based Detection] Ditemukan 7 baris anomali berdasarkan aturan logika.
           KODE                      NAMA_PRODUK                   LOKASI  \
136    B0000044                   BODREXIN DEMAM                      SYR   
151    B0000073                         BODREXIN  FB PE ANAK SYR (KERING)   
886    M0000185                    MIRASIC FORTE                    650MG   
1018   N0000139                     NIVEA CARE &      PROTECT SERUM 180ML   
1191  R00000102  RELIABLE COTTON BUDS DWS REFFIL                     60'S   

      QTY_STOK   UNIT  VALID_KODE        FLAG_ERROR  KODE_VALID  RULE_FLAG  
136          0   5,00        True  UNIT TIDAK VALID        True       True  
151          0   2,00        True  UNIT TIDAK VALID        True       True  
886          0  23,00        True  UNIT TIDAK VALID        True       True  
1018         0   1,00        True  UNIT TIDAK VALID        True       True  
1191         0   7,00        True  UNIT TIDAK VALID        True       True

#### CONSTRAINT-BASED DETECTION

In [15]:
# === [2] Constraint-Based Detection ===

# Definisikan batas dan validasi satuan yang wajar
max_qty = 10000
valid_units = ['STRIP', 'BOX', 'PCS', 'BTL', 'PACK', 'TUBE', 'SACHET']

# Deteksi pelanggaran constraint
df['CONSTRAINT_FLAG'] = np.where(
    (df['QTY_STOK'] < 0) |                                  # stok tidak boleh negatif
    (df['QTY_STOK'] > max_qty) |                            # stok terlalu besar
    (~df['UNIT'].isin(valid_units)) |                       # satuan tidak valid
    (df['LOKASI'].isna()) | (df['LOKASI'].str.strip() == '') |  # lokasi kosong
    (df['KODE'].str.len() < 3) | (df['KODE'].str.len() > 12),   # kode terlalu pendek/panjang
    True, False
)

# Hitung jumlah data melanggar constraint
constraint_count = df['CONSTRAINT_FLAG'].sum()

print(f"📏 [Constraint-Based Detection] Ditemukan {constraint_count} baris melanggar batas wajar.")

# Tampilkan contoh data yang bermasalah
if constraint_count > 0:
    print(df[df['CONSTRAINT_FLAG']].head(5))
else:
    print("✅ Tidak ditemukan pelanggaran constraint dalam dataset stok.")

print("--------------------------------------------------\n")

📏 [Constraint-Based Detection] Ditemukan 248 baris melanggar batas wajar.
       KODE        NAMA_PRODUK LOKASI  QTY_STOK  UNIT  VALID_KODE  \
10  A000069       ALKOHOL SWEP  ETL3B      1030   PSG        True   
11   A00007  AFITSON NO.1 18 G   ETL2        50   POT        True   
18   A00014   ALLETROL TM 5 ML   RAK4        60   FLS        True   
21  A000177  ANAK SUMANG 200'S   ETL1      1800  SACH        True   
27   A00023      AMOXSAN 500MG   RAK1       800   TAB        True   

          FLAG_ERROR  KODE_VALID  RULE_FLAG  CONSTRAINT_FLAG  
10  UNIT TIDAK VALID        True      False             True  
11  UNIT TIDAK VALID        True      False             True  
18  UNIT TIDAK VALID        True      False             True  
21  UNIT TIDAK VALID        True      False             True  
27  UNIT TIDAK VALID        True      False             True  
--------------------------------------------------



#### STATISTICAL / PATTERN-BASED DETECTION

In [16]:
# === [3] Statistical / Pattern-Based Detection (Outlier Detection) ===

# Fungsi deteksi outlier per produk berdasarkan Z-Score
def detect_outlier_group(df_group, col):
    mean = df_group[col].mean()
    median = df_group[col].median()
    std = df_group[col].std(ddof=0)

    # Jika data terlalu sedikit atau standar deviasi = 0, tidak dihitung sebagai outlier
    if len(df_group) < 3 or std == 0 or np.isnan(std):
        df_group[f'STAT_FLAG_{col}'] = False
        df_group[f'{col}_FIX_STAT'] = df_group[col]
        return df_group

    # Hitung Z-Score untuk mendeteksi nilai ekstrim (>3 SD dari rata-rata)
    zscore = (df_group[col] - mean) / std
    df_group[f'STAT_FLAG_{col}'] = abs(zscore) > 3
    df_group[f'{col}_FIX_STAT'] = np.where(abs(zscore) > 3, median, df_group[col])
    return df_group


# Terapkan fungsi per grup produk
df = df.groupby('NAMA_PRODUK', group_keys=False).apply(lambda g: detect_outlier_group(g, 'QTY_STOK'))

# Buat kolom indikator koreksi otomatis
df['DIKOREKSI_STAT'] = df['STAT_FLAG_QTY_STOK']

# Hitung total anomali
stat_anom_qty = df['STAT_FLAG_QTY_STOK'].sum()

print("📊 [Statistical Detection] (Per Produk)")
print(f"   Anomali QTY_STOK : {stat_anom_qty}")
print(f"✅ Jumlah data dikoreksi otomatis (statistik): {df['DIKOREKSI_STAT'].sum()}\n")

# Tampilkan contoh hasil koreksi
print("🔍 Contoh hasil koreksi outlier statistik:")
cols_show = [
    'KODE', 'NAMA_PRODUK', 'LOKASI',
    'QTY_STOK', 'QTY_STOK_FIX_STAT', 'STAT_FLAG_QTY_STOK'
]
print(df[df['DIKOREKSI_STAT']].head(10)[cols_show])

print("--------------------------------------------------\n")

📊 [Statistical Detection] (Per Produk)
   Anomali QTY_STOK : 0
✅ Jumlah data dikoreksi otomatis (statistik): 0

🔍 Contoh hasil koreksi outlier statistik:
Empty DataFrame
Columns: [KODE, NAMA_PRODUK, LOKASI, QTY_STOK, QTY_STOK_FIX_STAT, STAT_FLAG_QTY_STOK]
Index: []
--------------------------------------------------



C:\Users\hplam\AppData\Local\Temp\ipykernel_27608\4188820794.py:23: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('NAMA_PRODUK', group_keys=False).apply(lambda g: detect_outlier_group(g, 'QTY_STOK'))


#### CLUSTERING-BASED DETECTION

In [23]:
# ===============================================================
# 🧮 CLUSTERING-BASED DETECTION (Per Produk) + PENANGANAN OUTLIER
# ===============================================================

# 🧩 Pastikan semua kolom penting ada, deteksi nama fleksibel
qty_masuk_col = next((c for c in df.columns if 'QTY' in c and ('MSK' in c or 'MASUK' in c.upper())), None)
qty_keluar_col = next((c for c in df.columns if 'QTY' in c and ('KLR' in c or 'KELUAR' in c.upper())), None)
nilai_masuk_col = next((c for c in df.columns if 'NILAI' in c and ('MSK' in c or 'MASUK' in c.upper())), None)
nilai_keluar_col = next((c for c in df.columns if 'NILAI' in c and ('KLR' in c or 'KELUAR' in c.upper())), None)

# 📊 Hitung kolom total jika belum ada
if 'QTY_TOTAL' not in df.columns:
    if qty_masuk_col and qty_keluar_col:
        df['QTY_TOTAL'] = df[qty_masuk_col] - df[qty_keluar_col]
    else:
        df['QTY_TOTAL'] = df.select_dtypes(include='number').iloc[:, 0]

if 'NILAI_TOTAL' not in df.columns:
    if nilai_masuk_col and nilai_keluar_col:
        df['NILAI_TOTAL'] = df[nilai_masuk_col] - df[nilai_keluar_col]
    else:
        df['NILAI_TOTAL'] = df.select_dtypes(include='number').iloc[:, 1]

# 🧮 Inisialisasi kolom deteksi & koreksi
df = df.copy()
df.loc[:, 'CLUSTER_FLAG'] = False
df.loc[:, 'DIKOREKSI_CLUSTER'] = False
df.loc[:, 'QTY_TOTAL_FIX_CLUSTER'] = df['QTY_TOTAL']
df.loc[:, 'NILAI_TOTAL_FIX_CLUSTER'] = df['NILAI_TOTAL']

# ===============================================================
# 🔹 Fungsi deteksi + koreksi per produk
# ===============================================================
def clustering_per_produk(group):
    if len(group) < 4:
        return group

    X = group[['QTY_TOTAL', 'NILAI_TOTAL']].fillna(0)
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    # Buat 2 cluster per produk
    kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
    group = group.copy()
    group['CLUSTER'] = kmeans.fit_predict(X_scaled)

    centroids = kmeans.cluster_centers_
    distances = np.linalg.norm(X_scaled - centroids[group['CLUSTER']], axis=1)
    threshold = distances.mean() + 2 * distances.std()
    group['CLUSTER_FLAG'] = distances > threshold

    # Koreksi nilai ekstrem (outlier)
    for i, row in group[group['CLUSTER_FLAG']].iterrows():
        cid = row['CLUSTER']
        centroid_unscaled = scaler.inverse_transform([centroids[cid]])[0]
        group.at[i, 'QTY_TOTAL_FIX_CLUSTER'] = centroid_unscaled[0]
        group.at[i, 'NILAI_TOTAL_FIX_CLUSTER'] = centroid_unscaled[1]
        group.at[i, 'DIKOREKSI_CLUSTER'] = True

    return group

# ===============================================================
# 🔹 Terapkan fungsi ke setiap produk
# ===============================================================
if 'NAMA_PRODUK' in df.columns:
    df = df.groupby('NAMA_PRODUK', group_keys=False).apply(clustering_per_produk).reset_index(drop=True)
else:
    df = clustering_per_produk(df)

# ===============================================================
# 🔹 Laporan hasil deteksi & koreksi
# ===============================================================
outlier_count = int(df['CLUSTER_FLAG'].sum())
fixed_count = int(df['DIKOREKSI_CLUSTER'].sum())

print("🧮 [Clustering-Based Detection per Produk] "
      f"Ditemukan {outlier_count} data jauh dari pusat cluster produk.")
print(f"🩺 {fixed_count} data telah dikoreksi mendekati centroid cluster produk masing-masing.\n")

# ===============================================================
# 🔹 Tampilkan seluruh hasil tanpa potongan output
# ===============================================================
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

cols_show = [c for c in [
    'KODE', 'NAMA_PRODUK',
    'QTY_TOTAL', 'NILAI_TOTAL',
    'QTY_TOTAL_FIX_CLUSTER', 'NILAI_TOTAL_FIX_CLUSTER',
    'CLUSTER', 'CLUSTER_FLAG', 'DIKOREKSI_CLUSTER'
] if c in df.columns]

print("🔍 Contoh hasil koreksi clustering:")
print(df[df['DIKOREKSI_CLUSTER']][cols_show])
print("--------------------------------------------------\n")


🧮 [Clustering-Based Detection per Produk] Ditemukan 0 data jauh dari pusat cluster produk.
🩺 0 data telah dikoreksi mendekati centroid cluster produk masing-masing.

🔍 Contoh hasil koreksi clustering:
Empty DataFrame
Columns: [KODE, NAMA_PRODUK, QTY_TOTAL, NILAI_TOTAL, QTY_TOTAL_FIX_CLUSTER, NILAI_TOTAL_FIX_CLUSTER, CLUSTER_FLAG, DIKOREKSI_CLUSTER]
Index: []
--------------------------------------------------



C:\Users\hplam\AppData\Local\Temp\ipykernel_27608\650466344.py:66: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('NAMA_PRODUK', group_keys=False).apply(clustering_per_produk).reset_index(drop=True)


#### CROSS-DATASET CONSISTENCY

In [ ]:
# ===============================================================
# 🔗 CROSS-DATASET CONSISTENCY CHECK 
# ===============================================================

import numpy as np
import pandas as pd

# Simpan nilai sebelum perbaikan
df['NILAI_TOTAL_BEFORE_FIX'] = df['NILAI_TOTAL']

# --- Identifikasi nama kolom relevan secara otomatis ---
nilai_masuk_col = next((c for c in df.columns if 'NILAI' in c and ('MSK' in c or 'MASUK' in c.upper())), None)
nilai_keluar_col = next((c for c in df.columns if 'NILAI' in c and ('KLR' in c or 'KELUAR' in c.upper())), None)
kategori_col = next((c for c in df.columns if 'KATEGORI' in c.upper()), None)

# Jika kolom tidak ditemukan, buat nilai default agar tidak error
if nilai_masuk_col is None:
    df['NILAI_MSK'] = 0
    nilai_masuk_col = 'NILAI_MSK'

if nilai_keluar_col is None:
    df['NILAI_KLR'] = 0
    nilai_keluar_col = 'NILAI_KLR'

if kategori_col is None:
    df['KATEGORI'] = 'UNKNOWN'
    kategori_col = 'KATEGORI'

# --- Fungsi logika nilai seharusnya berdasarkan kategori ---
def nilai_seharusnya(row):
    kategori = str(row[kategori_col]).upper()
    if kategori == 'MASUK':
        return row[nilai_masuk_col]
    elif kategori == 'KELUAR':
        return row[nilai_keluar_col]
    else:
        # fallback: selisih antara masuk dan keluar
        return row[nilai_masuk_col] - row[nilai_keluar_col]

# Hitung nilai seharusnya berdasarkan kategori transaksi
df['NILAI_TOTAL_EXPECTED'] = df.apply(nilai_seharusnya, axis=1)

# Deteksi inkonsistensi awal
df['CONSIST_FLAG'] = abs(df['NILAI_TOTAL'] - df['NILAI_TOTAL_EXPECTED']) > 1e-6
before_fix = df['CONSIST_FLAG'].sum()

# Koreksi otomatis hanya untuk baris tidak konsisten
df.loc[df['CONSIST_FLAG'], 'NILAI_TOTAL'] = df.loc[df['CONSIST_FLAG'], 'NILAI_TOTAL_EXPECTED']
df['NILAI_TOTAL_FIX_CONSIST'] = df['NILAI_TOTAL']

# Recheck setelah perbaikan
df['CONSIST_FLAG_AFTER'] = abs(df['NILAI_TOTAL'] - df['NILAI_TOTAL_EXPECTED']) > 1e-6
after_fix = df['CONSIST_FLAG_AFTER'].sum()

# ===============================================================
# 🔹 Laporan hasil perbaikan konsistensi antar kolom
# ===============================================================
print(f"🔗 [Cross-Dataset Consistency] Sebelum koreksi: {before_fix} baris tidak konsisten.")
print(f"🧮 Setelah koreksi: {after_fix} baris masih tidak konsisten.")
print(f"✅ Berhasil memperbaiki {before_fix - after_fix} baris data otomatis.\n")

# ===============================================================
# 🔹 Contoh hasil koreksi (10 baris)
# ===============================================================
cols_show = [c for c in [
    'KODE', 'NAMA_PRODUK', kategori_col,
    nilai_masuk_col, nilai_keluar_col,
    'NILAI_TOTAL_BEFORE_FIX', 'NILAI_TOTAL_FIX_CONSIST'
] if c in df.columns]

print("🔍 Contoh hasil koreksi konsistensi nilai:")
print(df[df['CONSIST_FLAG_AFTER'] == False][cols_show].head(10))
print("--------------------------------------------------\n")


🔗 [Cross-Dataset Consistency] Sebelum koreksi: 1506 baris tidak konsisten.
🧮 Setelah koreksi: 0 baris masih tidak konsisten.
✅ Berhasil memperbaiki 1506 baris data otomatis.

🔍 Contoh hasil koreksi konsistensi nilai:
      KODE         NAMA_PRODUK KATEGORI  NILAI_MSK  NILAI_KLR  \
0  A000001          ANATON TAB  UNKNOWN          0          0   
1   A00001       ACTIVED HIJAU  UNKNOWN          0          0   
2  A000012  APIALYS SYR 100 ML  UNKNOWN          0          0   
3  A000014     ALKOHOL 1000 ML  UNKNOWN          0          0   
4  A000016     ALLOPURINOL 300  UNKNOWN          0          0   
5  A000018   ATORVASTATIN 10MG  UNKNOWN          0          0   
6   A00004     ACYCLOVIR 200MG  UNKNOWN          0          0   
7  A000040         MEFIX 500MG  UNKNOWN          0          0   
8   A00005     ACYCLOVIR 400MG  UNKNOWN          0          0   
9  A000066       ANDALAN KB FE  UNKNOWN          0          0   

   NILAI_TOTAL_BEFORE_FIX  NILAI_TOTAL_FIX_CONSIST  
0             

### Hasil final bersih 

In [45]:
# HASIL AKHIR FINAL BERSIH

import pandas as pd
from pathlib import Path
import datetime

# === 1. Baca dataset hasil olahan terakhir (dari CSV) ===
df = pd.read_csv("stok_final_fix.csv", encoding="utf-8-sig")

# === 2. Pilih kolom final sesuai dataset yang tersedia ===
cols_final = [
    'KODE', 'NAMA_PRODUK', 'UNIT',
    'QTY_TOTAL_FIX_STAT', 'NILAI_TOTAL_FIX_CONSIST'
]

# Pastikan hanya kolom yang ada di dataset yang diambil
cols_final = [col for col in cols_final if col in df.columns]

# === 3. Buat salinan data final ===
df_clean = df[cols_final].copy()

# === 4. Simpan ke file CSV ===
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = output_dir / f"dataset_final_bersih_{timestamp}.csv"

df_clean.to_csv(output_path, index=False, encoding='utf-8-sig', float_format='%.2f')

# === 5. Cetak ringkasan hasil ===
print(f"✅ Versi final bersih berhasil diekspor ke:\n📂 {output_path}")
print(f"📊 Total baris: {len(df_clean)} | Total kolom: {len(df_clean.columns)}")


✅ Versi final bersih berhasil diekspor ke:
📂 output\dataset_final_bersih_20251021_101658.csv
📊 Total baris: 1513 | Total kolom: 3
